In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# ==========================================
# 1. CONFIGURATION
# ==========================================
HA_URL = "http://homeassistant.local:8123"

# Go to the following url 'http://homeassistant.local:8123/profile/security' and look for Long-lived access tokens. Create one and add below 
TOKEN = "YOUR_TOKEN"

# Define your period here
START_DATE = "2025-01-01"
END_DATE = "2025-01-31" 

# ==========================================
# 2. HELPER FUNCTIONS
# ==========================================
def get_quatt_data(date_str):
    url = f"{HA_URL}/api/services/quatt/get_insights?return_response=true"
    headers = {
        "Authorization": f"Bearer {TOKEN}",
        "content-type": "application/json",
    }
    payload = {
        "from_date": date_str,
        "timeframe": "day",
        "advanced_insights": True
    }
    
    # We let the main loop handle exceptions to keep this simple
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()
    return response.json()

# ==========================================
# 3. MAIN EXECUTION LOOP
# ==========================================
start = datetime.strptime(START_DATE, "%Y-%m-%d")
end = datetime.strptime(END_DATE, "%Y-%m-%d")
current_date = start

hourly_chunks = []
daily_records = []

print(f"Starting data collection from {START_DATE} to {END_DATE}...")

while current_date <= end:
    date_str = current_date.strftime("%Y-%m-%d")
    print(f"Fetching data for: {date_str}...", end=" ")
    
    try:
        # 1. Call API
        json_response = get_quatt_data(date_str)
        
        # 2. Validate Response Structure
        if not json_response or 'service_response' not in json_response:
            raise ValueError("Missing 'service_response' in API output")
            
        data = json_response['service_response']
        
        # 3. Prepare the base daily record
        # We use .get(key, 0) for numeric fields to prevent NoneType errors later
        daily_record = {
            "date": date_str,
            "totalHpHeat": data.get("totalHpHeat", 0), 
            "totalHpElectric": data.get("totalHpElectric", 0),
            "totalBoilerHeat": data.get("totalBoilerHeat", 0),
            "totalBoilerGas": data.get("totalBoilerGas", 0),
            "averageCOP": data.get("averageCOP"),
            "savingsMoney": data.get("savingsMoney"),
            "savingsCo2": data.get("savingsCo2"),
            "savingsGas": data.get("savingsGas"),
            "savingsGasMoney": data.get("savingsGasMoney"),
            "savingsQuattElectricityCost": data.get("savingsQuattElectricityCost"),
            "co2GasSaved": data.get("co2GasSaved"),
            "co2Electricity": data.get("co2Electricity"),
            "hasTarrifs": data.get("hasTarrifs")
        }

        # 4. Process Hourly Data
        df_main = pd.DataFrame(data.get('graph', []))
        df_out = pd.DataFrame(data.get('outsideTemperatureGraph', []))
        df_water = pd.DataFrame(data.get('waterTemperatureGraph', []))
        df_room = pd.DataFrame(data.get('roomTemperatureGraph', []))

        if not df_main.empty:
            # Merge hourly data
            df_merged = df_main.merge(df_out, on='timestamp', how='outer')
            df_merged = df_merged.merge(df_water, on='timestamp', how='outer')
            df_merged = df_merged.merge(df_room, on='timestamp', how='outer')
            
            # Calculate averages inside the loop
            cols_to_avg = ['temperatureOutside', 'waterTemperature', 'roomTemperature', 'roomSetpoint']
            existing_cols = [c for c in cols_to_avg if c in df_merged.columns]
            
            if existing_cols:
                averages = df_merged[existing_cols].mean().to_dict()
                for key, value in averages.items():
                    daily_record[f"avg_{key}"] = value

            hourly_chunks.append(df_merged)
            print("Success.")
        else:
            print("Daily stats found, but no hourly graph data.")
        
        # Append the successfully processed record
        daily_records.append(daily_record)

    except Exception as e:
        # This block catches API errors, Parsing errors, or Data errors
        print(f"SKIPPED. Error: {e}")
        # We do NOT raise the error, allowing the loop to continue
    
    # Move to next day regardless of success/failure
    current_date += timedelta(days=1)
    time.sleep(0.2) 

# ==========================================
# 4. FINAL AGGREGATION
# ==========================================

# Build Hourly DataFrame
if hourly_chunks:
    df_hourly = pd.concat(hourly_chunks, ignore_index=True)
    df_hourly['timestamp'] = pd.to_datetime(df_hourly['timestamp'])
    df_hourly = df_hourly.sort_values('timestamp').set_index('timestamp')
else:
    df_hourly = pd.DataFrame()

# Build Daily DataFrame
if daily_records:
    df_daily = pd.DataFrame(daily_records)
    df_daily['date'] = pd.to_datetime(df_daily['date'])
    df_daily = df_daily.sort_values('date').set_index('date')
    
    # Calculate totalHeatPerHour
    # We fillna(0) to ensure math doesn't fail if a column is missing
    df_daily['totalHeatPerHour'] = (
        df_daily.get('totalHpHeat', 0) + df_daily.get('totalBoilerHeat', 0)
    ) / 24
else:
    df_daily = pd.DataFrame()

print("\nData Collection Complete!")
print(f"Hourly Data Rows: {len(df_hourly)}")
print(f"Daily Data Rows:  {len(df_daily)}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd

# Check if dataframe exists
if 'df_daily' in locals() and not df_daily.empty:
    
    # 1. PREPARE AND CLEAN DATA
    plot_data = df_daily[['avg_temperatureOutside', 'totalHeatPerHour', 'averageCOP']].copy()
    plot_data = plot_data.replace([np.inf, -np.inf], np.nan).dropna()

    if len(plot_data) >= 2:
        
        # 2. Define X, Y, and Colors
        x = plot_data['avg_temperatureOutside']
        y = plot_data['totalHeatPerHour']
        colors = plot_data['averageCOP']
        
        # 3. Setup Figure and Axis
        plt.figure(figsize=(12, 7))
        ax = plt.gca()
        
        # 4. Create Scatter Plot with Color Scale
        sc = plt.scatter(x, y, c=colors, cmap='viridis', s=100, alpha=0.9, edgecolors='black', zorder=3)
        
        # Add Colorbar
        cbar = plt.colorbar(sc)
        cbar.set_label('Average COP', fontsize=12)
        
        # 5. Add Extrapolated Trendline & Calculate Intercepts
        try:
            z = np.polyfit(x, y, 1) 
            p = np.poly1d(z)        
            
            # Extrapolate from -10 up to the max temp
            x_range = np.linspace(-10, x.max(), 100)
            
            plt.plot(x_range, p(x_range), "r--", linewidth=2, label='Stooklijn (Extrapolated)', zorder=2)
            
            # --- CALCULATIONS ---
            slope = z[0]
            intercept = z[1]
            
            # Calculate X-Intercept (Where y = 0)
            # 0 = slope * x + intercept  =>  x = -intercept / slope
            if slope != 0:
                x_intercept = -intercept / slope
            else:
                x_intercept = 0

            print("-" * 40)
            print(f"Trendline Equation: Heat = {slope:.2f} * Temp + {intercept:.2f}")
            print(f"Theoretical Heat Demand at -10°C: {p(-10):.0f} Wh/h")
            print(f"X-Axis Intercept (Zero Heat Needed): {x_intercept:.2f} °C")
            print("-" * 40)

        except Exception as e:
            print(f"Could not calculate trendline: {e}")

        # 6. Formatting & Ticks
        plt.title('Heating Demand vs Outside Temperature (Colored by COP)', fontsize=16)
        plt.xlabel('Average Outside Temperature (°C)', fontsize=12)
        plt.ylabel('Total Heat Per Hour (Wh)', fontsize=12)
        
        # Minor ticks configuration
        ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))    # Every 1 degree
        ax.yaxis.set_minor_locator(ticker.MultipleLocator(1000)) # Every 1000 Wh
        
        # Grid Settings
        plt.grid(True, which='major', linestyle='--', alpha=0.8, color='gray')
        plt.grid(True, which='minor', linestyle=':', alpha=0.4, color='gray')

        plt.legend(loc='upper right')
        plt.xlim(left=-10.5)
        
        # Optional: Set Y limit to 0 so the graph doesn't float if data is high
        plt.ylim(bottom=0) 
        
        plt.show()
        
    else:
        print("Not enough valid data points to plot.")
else:
    print("Dataframe 'df_daily' not found.")